In [1]:
pip install wbdata pandas sqlalchemy pyodbc


  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging
from datetime import datetime

# Configure logging
log_filename = f"elt_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(filename=log_filename,
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("ELT script started")


In [8]:
#testing API connectivity
import requests

url = "http://api.worldbank.org/v2/country?incomeLevel=HIC&format=json"
response = requests.get(url)

print("Status code:", response.status_code)
print("Content:", response.text[:500])  # Print part of response


Status code: 200
Content: [{"page":1,"pages":2,"per_page":"50","total":85},[{"id":"ABW","iso2Code":"AW","name":"Aruba","region":{"id":"LCN","iso2code":"ZJ","value":"Latin America & Caribbean "},"adminregion":{"id":"","iso2code":"","value":""},"incomeLevel":{"id":"HIC","iso2code":"XD","value":"High income"},"lendingType":{"id":"LNX","iso2code":"XX","value":"Not classified"},"capitalCity":"Oranjestad","longitude":"-70.0167","latitude":"12.5167"},{"id":"AND","iso2Code":"AD","name":"Andorra","region":{"id":"ECS","iso2code":"


In [9]:
import wbdata
import pandas as pd
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Retrieve countries with High Income level
    countries = wbdata.get_country(incomelevel="HIC")
    country_codes = [c['id'] for c in countries]

    # Define the indicator
    indicator = {'GC.DOD.TOTL.GD.ZS': 'DebtPercentGDP'}

    # Set the date range
    data_date = (datetime(2000, 1, 1), datetime(2020, 1, 1))

    # Fetch the data
    df = wbdata.get_dataframe(indicator, country=country_codes, data_date=data_date)
    print(df.head())

except Exception as e:
    logger.error(f"An error occurred: {e}")


In [10]:
import logging

# Configure logging
logging.basicConfig(
    filename='data_extraction.log',
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)

logging.info('Data extraction started.')

# Your data extraction code here

logging.info('Data extraction completed successfully.')


In [11]:
import logging
from datetime import datetime

# Set up logging
logging.basicConfig(
    filename="extract_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("=== Extract Step Started ===")


In [12]:
import wbdata
import pandas as pd

def extract_debt_data(start_year=2010, end_year=2023):
    try:
        logging.info("Attempting to fetch high income countries...")
        countries = wbdata.get_countries(incomelevel="HIC")  # High income
        logging.info("Successfully fetched high income countries.")
    except Exception as e:
        logging.warning(f"Failed to get high income countries: {e}")
        logging.info("Falling back to all countries (excluding aggregates).")
        countries = wbdata.get_countries()

    # Filter country codes (exclude aggregates)
    country_codes = [c['id'] for c in countries if c['region']['id'] != 'NA']
    logging.info(f"Fetched {len(country_codes)} valid country codes.")

    # Define indicator: Central government debt (% of GDP)
    indicator = {"GC.DOD.TOTL.GD.ZS": "DebtPercentGDP"}

    try:
        data = wbdata.get_dataframe(
            indicator,
            country=country_codes,
            data_date=(datetime(start_year, 1, 1), datetime(end_year, 1, 1)),
            convert_date=True
        )
        logging.info(f"Successfully extracted data for years {start_year}-{end_year}.")
        return data.reset_index()
    except Exception as e:
        logging.error(f"Data extraction failed: {e}")
        return pd.DataFrame()


In [13]:
import pyodbc

conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=elt-world-bank.database.windows.net;"
    "DATABASE=ELT;"
    "UID=CloudSA648a5ceb;"
    "PWD=Urno9@$$;"
)

try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connection successful!")
except Exception as e:
    print("❌ Connection failed:")
    print(e)


❌ Connection failed:
('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: Timeout error [258].  (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Unable to complete login process due to delay in login response (258)')


In [14]:
import wbdata
import pandas as pd

def extract_debt_percent_gdp(start_year=2010, end_year=2023):
    try:
        logging.info("Attempting to fetch high income countries...")
        countries = wbdata.get_countries(incomelevel="HIC")  # High income
        logging.info("Successfully fetched high income countries.")
    except Exception as e:
        logging.warning(f"Failed to get high income countries: {e}")
        logging.info("Falling back to all countries (excluding aggregates).")
        countries = wbdata.get_countries()

    # Filter country codes (exclude aggregates)
    country_codes = [c['id'] for c in countries if c['region']['id'] != 'NA']
    logging.info(f"Fetched {len(country_codes)} valid country codes.")

    # Define indicator: Central government debt (% of GDP)
    indicator = {"GC.DOD.TOTL.GD.ZS": "DebtPercentGDP"}

    try:
        data = wbdata.get_dataframe(
            indicator,
            country=country_codes,
            data_date=(datetime(start_year, 1, 1), datetime(end_year, 1, 1)),
            convert_date=True
        )
        logging.info(f"Successfully extracted data for years {start_year}-{end_year}.")
        return data.reset_index()
    except Exception as e:
        logging.error(f"Data extraction failed: {e}")
        return pd.DataFrame()


In [15]:
# debt_data_extraction.py

import wbgapi as wb
import pandas as pd
import logging
from datetime import datetime
import os

# Setup logging
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, "debt_data_extraction.log")
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def extract_debt_data():
    try:
        logging.info("Starting debt data extraction")

        # Set database (WDI)
        wb.db = 2

        # Indicator for Central government debt, % of GDP
        indicator = 'GC.DOD.TOTL.GD.ZS'

        # Get High Income Country members
        countries = wb.region.members('HIC')

        # Get data (entire time range available)
        df = wb.data.DataFrame(indicator, economy=countries, labels=True, time=range(2000, 2024))

        # Optional: Clean column names
        df.reset_index(inplace=True)
        df.rename(columns={
            'economy': 'Country Code',
            'country': 'Country',
            'time': 'Year',
            indicator: 'Debt (% of GDP)'
        }, inplace=True)

        # Save data to CSV
        output_dir = "data"
        os.makedirs(output_dir, exist_ok=True)
        filename = f"debt_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(os.path.join(output_dir, filename), index=False)

        logging.info(f"Data extraction completed successfully. File saved as {filename}")

    except Exception as e:
        logging.error(f"Error during data extraction: {str(e)}")

# Run
extract_debt_data()


In [16]:
!pip install schedule


In [ ]:
import schedule
import time

# Schedule to run once daily
schedule.every().day.at("09:00").do(extract_debt_data)

while True:
    schedule.run_pending()
    time.sleep(60)